In [19]:
import pandas as pd
import numpy as np

In [20]:
dataset = pd.read_csv('project3_dataset4.txt', sep="\t", header=None)
nrows = len(dataset)
ncol = len(dataset.columns)
dataset= np.array(dataset)

[['sunny' 'hot' 'high' 'weak' 0]
 ['sunny' 'hot' 'high' 'strong' 0]
 ['overcast' 'hot' 'high' 'weak' 1]
 ['rain' 'mild' 'high' 'weak' 1]
 ['rain' 'cool' 'normal' 'weak' 1]
 ['rain' 'cool' 'normal' 'strong' 0]
 ['overcast' 'cool' 'normal' 'strong' 1]
 ['sunny' 'mild' 'high' 'weak' 0]
 ['sunny' 'cool' 'normal' 'weak' 1]
 ['rain' 'mild' 'normal' 'weak' 1]
 ['sunny' 'mild' 'normal' 'strong' 1]
 ['overcast' 'mild' 'high' 'strong' 1]
 ['overcast' 'hot' 'normal' 'weak' 1]
 ['rain' 'mild' 'high' 'strong' 0]]


In [21]:
def check_termination(data):
    global ncol
    if len(np.unique(data[:,ncol-1]))==1:
        return True
    else:
        return False                                       

In [36]:
def classify(data):
    global ncol
    classes, counts= np.unique(data[:,ncol-1], return_counts=True)
    return classes[counts.argmax()]

In [23]:
def Get_Acc_new(Y,target):
    Y =np.array(Y)
    target=np.array(target)
    accuracy = 0.0
    counter = 0
    for i in range(len(Y)):
            if Y[i] == target[i]:
                counter= counter+1
    accuracy = (float((counter)*100)/float(len(Y)))
    return accuracy

In [24]:
def get_splits(data):
    global ncol
    global FEATURE_TYPES
    potential_splits = {}
    for i in range(ncol-1):          # excluding the last column which is the label
        unique_values = np.unique(data[:,i])
        if FEATURE_TYPES[i] == "continuous":
            potential_splits[i] = []
            for j in range(1,len(unique_values)):
                    potential_split = (unique_values[j] + unique_values[j- 1]) / 2
                    potential_splits[i].append(potential_split)
        else:
            potential_splits[i] = unique_values
    
    return potential_splits

In [25]:
def split_data(data, column, svalue):
        left= data[data[:,column] <= svalue]
        right= data[data[:,column] >  svalue]  
        return left,right

In [26]:
def change_feature(dataset):
    global FEATURE_TYPES
    for i in range(len(FEATURE_TYPES)):
        if FEATURE_TYPES[i] == "categorical":
            values = np.unique(dataset[:,i]).tolist()
            for j in range(len(dataset[:,i])):
                for k in values:
                    if dataset[j][i] == k:
                        dataset[j][i] = values.index(k)
    return dataset

In [27]:
def feature_types(df):
    feature_types = []
    for i in range(len(df[0])-1):
            value = df[0][i]
            if (isinstance(value, str)):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    return feature_types

In [28]:
def gini(data):
    global ncol
    classes, counts = np.unique(data[:,ncol-1], return_counts=True)
    prob = counts / counts.sum()
    gini = 1 - np.sum(np.power(prob,2))
    return gini


In [29]:
def overall_gini_func(left,right):
    n = len(left) + len(right)
    p_left = len(left) / n
    p_right = len(right) / n
    overall_gini =  (p_left * gini(left) 
                      + p_right *gini(right))
    
    return overall_gini

In [30]:
def determine_best_split(data, potential_splits):
    overall_gini = 1
    for i in range(len(data[0])-1):
        for value in potential_splits[i]:
            left,right = split_data(data, column = i,svalue = value)
            current_overall_gini = overall_gini_func(left,right)
            if current_overall_gini <= overall_gini:
                overall_gini = current_overall_gini
                best_split_column = i
                best_split_value = value
    return best_split_column, best_split_value

In [31]:
def best_split(data):
    overall_gini = 1
    global means 
    for i in range(len(data[0])-1):
            left,right = split_data(data, column = i,svalue = means[i])
            current_overall_gini = overall_gini_func(left,right)
            if current_overall_gini <= overall_gini:
                overall_gini = current_overall_gini
                best_split_column = i
                best_split_value = means[i]
    return best_split_column, best_split_value

In [14]:
print(determine_best_split(dataset,get_splits(dataset)))

NameError: name 'FEATURE_TYPES' is not defined

In [32]:
def decision_tree(data, depth=0, min_samples=2, max_depth=5):
    
    if (check_termination(data)) or (len(data) < min_samples) or (depth == max_depth):
        return classify(data)
    else:    
        depth+= 1
        potential_splits = get_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        left,right = split_data(data, split_column, split_value)
        condition = "{} <= {}".format(split_column, split_value)
        sub_tree = {condition: []}
        one = decision_tree(left, depth, min_samples, max_depth)
        zero = decision_tree(right, depth, min_samples, max_depth)
        if one == zero:
            sub_tree = one
        else:
            sub_tree[condition].append(one)
            sub_tree[condition].append(zero)
        
        return sub_tree

In [33]:
def decision_tree_new(data, depth=0, min_samples=2, max_depth=5):
    global means
    if (check_termination(data)) or (len(data) < min_samples) or (depth == max_depth):
        return classify(data)
    else:    
        depth+= 1
        #potential_splits = get_splits(data)
        split_column, split_value = best_split(data)
        left,right = split_data(data, split_column, split_value)
        condition = "{} <= {}".format(split_column, split_value)
        sub_tree = {condition: []}
        one = decision_tree(left, depth, min_samples, max_depth)
        zero = decision_tree(right, depth, min_samples, max_depth)
        if one == zero:
            sub_tree = one
        else:
            sub_tree[condition].append(one)
            sub_tree[condition].append(zero)
        
        return sub_tree

In [37]:
FEATURE_TYPES = feature_types(dataset)
print(FEATURE_TYPES)
COLUMN_NUMBERS = [i for i in range(len(dataset[0]))]
#dataset1 = change_feature(dataset)
means =[]
for i in range(len(dataset1[0])-1):
    sum_r=0
    if FEATURE_TYPES[i] == "continuous":
        for j in dataset[:,i]:
            sum_r = sum_r+j
        avg = sum_r/len(dataset1)
        means.append(avg)
    else:
        means.append(0)
print(means)      
tree = decision_tree(dataset)
print(tree)

['categorical', 'categorical', 'categorical', 'categorical']
[0, 0, 0, 0]
{'0 <= overcast': [1, {'2 <= high': [{'0 <= rain': [{'3 <= strong': [0, 1]}, 0]}, {'3 <= strong': [{'1 <= cool': [0, 1]}, 1]}]}]}


In [ ]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")
    # ask question
    if comparison_operator == "<=":
        if example[int(feature_name)] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [ ]:
example = dataset1
classes1 =[]
for i in example:
    classes1.append(classify_example(i,tree))  
print(classes1)

In [ ]:
Get_Acc_new(classes1,dataset[:,ncol-1])